In [5]:
import os

os.sys.path.insert(0, 'src/')
import pdb_numpy
from pdb_numpy.analysis import compute_pdockQ

In [6]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [7]:
file = "src/pdb_numpy/tests/input/1rxz_11587_unrelaxed_rank_001_alphafold2_multimer_v3_model_4_seed_000.pdb"
test = pdb_numpy.Coor(file)


In [8]:
%timeit test = pdb_numpy.Coor(file)

13.1 ms ± 202 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
%lprun -f pdb_numpy.format.pdb.parse pdb_numpy.Coor(file)

Timer unit: 1e-09 s

Total time: 0.0442948 s
File: /home/murail/Documents/Code/pdb_numpy/src/pdb_numpy/format/pdb.py
Function: parse at line 22

Line #      Hits         Time  Per Hit   % Time  Line Contents
    22                                           def parse(pdb_lines, pqr_format=False):
    23                                               """Parse the pdb lines and return atom information's as a dictionary
    24                                           
    25                                               Parameters
    26                                               ----------
    27                                               pdb_lines : list
    28                                                   list of pdb lines
    29                                               pqr_format : bool, optional
    30                                                   if True, parse pqr format, by default False
    31                                           
    32                    

In [9]:
%timeit test.select_atoms("chain A and name CA")

89.1 µs ± 2 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [10]:
test_2 = test.select_atoms("chain A and name CA")
test_2.chain[:] = 'z'

In [11]:
import numpy as np
np.unique(test.chain)

array(['A', 'B'], dtype='<U2')

In [12]:
test.get_aa_seq()

{'A': 'MIDVIMTGELLKTVTRAIVALVSEARIHFLEKGLHSRAVDPANVAMVIVDIPKDSFEVYNIDEEKTIGVDMDRIFDISKSISTKDLVELIVEDESTLKVKFGSVEYKVALIDPSAIRKEPRIPELELPAKIVMDAGEFKKAIAAADKISDQVIFRSDKEGFRIEAKGDVDSIVFHMTETELIEFNGGEARSMFSVDYLKEFCKVAGSGDLLTIHLGTNYPVRLVFELVGGRAKVEYILAPRIESE',
 'B': 'KSTQATLERWF'}

In [13]:
%timeit test.get_aa_DL_seq()

59.9 ms ± 1.43 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
%timeit test.get_aa_seq()

761 µs ± 7.07 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [15]:
#%lprun -f compute_pdockQ compute_pdockQ(test)

In [16]:
%lprun -f pdb_numpy.Model.simple_select_atoms compute_pdockQ(test)

Timer unit: 1e-09 s

Total time: 0.000542098 s
File: /home/murail/Documents/Code/pdb_numpy/src/pdb_numpy/model.py
Function: simple_select_atoms at line 225

Line #      Hits         Time  Per Hit   % Time  Line Contents
   225                                               def simple_select_atoms(self, column, values, operator="=="):
   226                                                   """Select atoms from the PDB file based on the selection tokens.
   227                                                   Selection tokens are simple selection containing only one
   228                                                   keyword, operator, and values.
   229                                           
   230                                                   The keywords :
   231                                           
   232                                                   - `"resname"`
   233                                                   - `"chain"`
   234                      